# Debug model predictions

In this notebook we're going to explore methods to debug a machine-learning model using explainers. We'll cover the following topics:

* Breaking down predictions
* What-if scenarios

If you haven't read the README.md file yet, we encourage that you read that document first. It provides an important introduction to the use-case and dataset used in this notebook and other notebooks in the project.

Let's start by preparing everything we need for this notebook.

## Preparation steps

Before we can use an explainer on our model, we need to load a test dataset and the model itself. The model for this demo is based on the [UCI Credit card defaulters dataset](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients). You can learn more about training the model in `tasks/train-model.py`. For more details on the data preparation you can check out the `tasks/prepare-data.py` and `tasks/split-data.py` scripts.

Let's start by loading up the necessary Python packages:

In [1]:
import joblib
import pandas as pd

After loading the libraries for the model, let's load the model and some test data.
You can use `df_test.head()` to see what's in the dataset if you want.

In [2]:
model = joblib.load('../models/classifier.bin')
df_test = pd.read_csv('../data/processed/test.csv').sample(385)

x_test = df_test.drop('LABEL', axis=1)
y_test = df_test['LABEL']

With the dataset and model ready to go, let's begin calculating feature importance information for the model.

## Breaking down predictions

First, we're going to take a look at prediction-level explanations using the [iBreakDown algorithm](https://arxiv.org/pdf/1903.11420.pdf). This algorithm explains how the model came to a prediction by following the features from most important to least important. This shows you exactly what reasoning the model is following to come to a certain conclusion. Using this method you can determine which value could be causing a wrong prediction. This technique is also useful to explain to your customer how you came to a decision. Keep in mind though, the explainer is rather technical so it requires extra explanation if you're going to offer the explanation to your customer.

Let's take a look at what the iBreakdown explainer can do.

In [3]:
import dalex as dx

In [4]:
explainer = dx.Explainer(model, x_test, y_test, model_type='classifier', label='LABEL')

Preparation of a new explainer is initiated

  -> data              : 385 rows 29 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 385 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : LABEL
  -> predict function  : <function yhat_proba_default at 0x0000013FED648C10> will be used (default)
  -> predict function  : Accepts pandas.DataFrame and numpy.ndarray.
  -> predicted values  : min = 0.0, mean = 0.233, max = 0.87
  -> model type        : classifier will be used
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.835, mean = -0.00968, max = 0.98
  -> model_info        : package sklearn

A new explainer has been created!


We're going to break down the prediction to see how the model came to make its prediction. First, let's look at the actual outcome of the model:

In [9]:
y_test.iloc[[1]]

2178    0
Name: LABEL, dtype: int64

Then, we can use the `explainer` object to get the explanation for a single prediction:

In [5]:
explainer.predict_parts(x_test.iloc[[1]], type='break_down_interactions').plot()

The explainer breaks down the prediction for each individual feature. You can follow the features from top to bottom to get a sense of what each feature contributes to the outcome of the model.

As you can see, this is a great explainer for data scientists to debug their model. You can check the contribution of each feature to the outcome. Anything that stands out in this picture will help you find data errors or training problems.

Let's continue exploring the structure of the model by diving into the features a bit more. 

## What-if scenarios

In the previous section we've covered how to break down a prediction. In this section we're taking a look at what-if scenarios for predictions. You can create a what-if scenario using the `predict_profile` method on the explainer. 

In [6]:
explainer.predict_profile(x_test.iloc[[0]]).plot(variables=['PAY_1'])

Calculating ceteris paribus: 100%|██████████| 29/29 [00:00<00:00, 83.90it/s]


The plot shows the relationship between the value of `PAY_1` and the outcome of the model. On the x-axis we can see the values for the `PAY_1` feature. The y-axis shows the output value of the model. There's a dot on the plot that shows the value for `PAY_1` used in the prediction.

This plot shows what would happen if you changed the value for `PAY_1` leaving all other values unmodified. For example,if we were to change the value for `PAY_1` to 2, the output of the model would go up to 0.35 thus making the customer more likely to default.

Using the `predict_profile` method can help you gain a better understanding of what needs to change in the input to change the output of the model. 

## Summary

In this notebook we've explored how to use explainers to debug machine learning models using explainers by looking at prediction-level explanations.

I hope you enjoyed this series of notebooks on Explainable AI. If you have any questions, contact me on twitter: @willem_meints.